In [ ]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

In [5]:
"""Load asdiv"""

if model_name == "gpt-j":
    priming_text_path = "data/priming_texts/asdiv/asdiv_prefix.txt" # for gpt-j
else:
    priming_text_path = "data/priming_texts/asdiv/asdiv_prefix_codegen.txt" # for codegen

current_dataset = dh.init_dataset_from_name("asdiv", primingtext_path = priming_text_path)

---

In [1]:
import os
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from termcolor import colored
import wandb
import importlib

/home/CodeGen_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import utils.dataset_handler as dh
import utils.loading_utils as lu
import utils.testing_utils as tu

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

#model_name = "gpt-j"
model_name = "codegen"

/home/CodeGen_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if model_name == "gpt-j":
    """GPT-J and codeparrot models run in HFTest venv"""
    tokenizer = AutoTokenizer.from_pretrained(gptj_model)
    model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()
elif model_name == "codegen":
    """CodeGen runs in the venv venv"""
    model_args = lu.model_args()
    #model_args.model = "codegen-350M-mono"
    model, tokenizer = lu.load_CodeGen(model_args)

loading parameters
loading parameters took 434.02s
loading tokenizer
loading tokenizer took 3.93s


In [5]:
import importlib
from termcolor import colored
#import exp_impl.eq_legacy.func_def_eq_short as exp_impl
import exp_impl.func_def_general as exp_impl


#priming_text_path = "data/priming_texts/gsm8k/clustering_prompt/3_clusters/cluster_2.txt"  # for codegen
#priming_text_path = "data/priming_texts/gsm8k/codegen/func_eq_short.txt"
priming_text_path = "data/priming_texts/gsm8k/codegen/func_short.txt"
#priming_text_path = "data/priming_texts/gsm8k/concepts_prompt/part-whole_3.txt"
#wandb_run_name = "@100-codegen-0"
importlib.reload(exp_impl)

"""Load gsm8k"""

if model_name == "gpt-j":
    priming_text_path = (
        "data/priming_texts/gsm8k/gpt-j/gsm8k_fewer_alt.txt"  # for gpt-j
    )
    current_dataset = dh.init_dataset_from_name(
        "gsm8k", primingtext_path=priming_text_path
    )
else:
    current_dataset = dh.init_dataset_from_name(
        "gsm8k",
        primingtext_path=priming_text_path,
        sample_func=exp_impl.sample_n_for_prompting,
        generate_prompt_func=exp_impl.generate_prompt,
    )

tu.set_all_seeds()
#tu.set_all_seeds_alt()

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(100, inc_eq=False)

with open("test_prompt_gen.txt", "w") as f:
    f.write(current_dataset.generate_prompt(sample_q_list[0]))

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

def exercise4():
    """
    Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days?
    """
80


In [6]:
# Set up for CodeGen
config = lu.codegen_gen_args()
#config.num_return_sequences = 4 # 4 for gsm8k 5 for asdiv
config.num_return_sequences = 4
config.k = 3
config.max_lenght_after_input = 250
#config.top_p = 0.95
config.top_p = 0.95
config.top_k = 50
#config.temperature = 0.7
config.temperature = 0.61
config.min_length = 3

tu.set_all_seeds(model_name)
_, pass_at_k_list = tu.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, func_def_mod=True, print_output=False)

TESTING STARTED
@sample 5 -> Pass@3 = 0.35
@sample 10 -> Pass@3 = 0.275
@sample 15 -> Pass@3 = 0.18333333333333332
@sample 20 -> Pass@3 = 0.175


KeyboardInterrupt: 

In [ ]:
with open(f"results_lists/{priming_text_path}.pkl", "wb") as f:
    pickle.dump(pass_at_k_list, f)

In [10]:
print(colored(f"Transfer prompts - Pass@{3} = {np.mean(np.array(general_pass_at_k))}", "green"))

Transfer prompts - Pass@3 = 0.32899999999999996


In [17]:
print(colored(f"Transfer prompts - Pass@{3} = {np.mean(np.array(cluster_0_pass_at_k))}", "green"))
print(colored(f"Transfer prompts - Pass@{3} = {np.mean(np.array(cluster_1_pass_at_k))}", "green"))
print(colored(f"Transfer prompts - Pass@{3} = {np.mean(np.array(cluster_2_pass_at_k))}", "green"))

Transfer prompts - Pass@3 = 0.36399999999999993
Transfer prompts - Pass@3 = 0.338
Transfer prompts - Pass@3 = 0.391


In [18]:
import pickle

with open("clustering_exp/3_cluster/cluster_0_pass_at_k.pkl", "wb") as f:
    pickle.dump(cluster_0_pass_at_k, f)
with open("clustering_exp/3_cluster/cluster_1_pass_at_k.pkl", "wb") as f:
    pickle.dump(cluster_1_pass_at_k, f)
with open("clustering_exp/3_cluster/cluster_2_pass_at_k.pkl", "wb") as f:
    pickle.dump(cluster_2_pass_at_k, f)

In [16]:
print(colored(f"Part-whole promts - Pass@{3} = {np.mean(np.array(part_whole_pass_at_k))}", "green"))

NameError: name 'part_whole_pass_at_k' is not defined

In [25]:
print(colored(f"Transfer prompts - Pass@{3} = {np.mean(np.array(transfer_pass_at_k))}", "green"))
print(colored(f"Dimansional analysis promts - Pass@{3} = {np.mean(np.array(dimension_analysis_pass_at_k))}", "green"))
print(colored(f"Explicit math promts - Pass@{3} = {np.mean(np.array(explicit_math_pass_at_k))}", "green"))
print(colored(f"Part-whole promts - Pass@{3} = {np.mean(np.array(part_whole_pass_at_k))}", "green"))
print(colored(f"General prompts - Pass@{3} = {np.mean(np.array(general_pass_at_k))}", "green"))

Transfer prompts - Pass@3 = 0.345
Dimansional analysis promts - Pass@3 = 0.318
Explicit math promts - Pass@3 = 0.37799999999999995
Part-whole promts - Pass@3 = 0.41800000000000004


NameError: name 'general_pass_at_k' is not defined

In [26]:
import pickle

with open("transfer_pass_at_k.pkl", "wb") as f:
    pickle.dump(transfer_pass_at_k, f)
with open("dimension_analysis_pass_at_k.pkl", "wb") as f:
    pickle.dump(dimension_analysis_pass_at_k, f)
with open("explicit_math_pass_at_k.pkl", "wb") as f:
    pickle.dump(explicit_math_pass_at_k, f)
with open("part_whole_pass_at_k.pkl", "wb") as f:
    pickle.dump(part_whole_pass_at_k, f)

In [ ]:
# Set up for gpt-j
#config = lu.gptj_gen_args()

#tu.set_all_seeds(model_name)
#hf.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, print_output=False)

In [30]:
with wandb.init(project="PracticalWork", entity="antoniolopardo",config=config, name=wandb_run_name):

        tu.set_all_seeds(model_name)
        pass_at_k, _ = tu.testing_loop(current_dataset, tokenizer, model, sample_q_list, sample_a_list, config, func_def_mod=True, print_output=False)

        wandb.log({"pass_at_k": pass_at_k})

TESTING STARTED
@sample 5 -> Pass@3 = 0.0
@sample 10 -> Pass@3 = 0.0
@sample 15 -> Pass@3 = 0.0
@sample 20 -> Pass@3 = 0.0
@sample 25 -> Pass@3 = 0.0
@sample 30 -> Pass@3 = 0.0
@sample 35 -> Pass@3 = 0.0
@sample 40 -> Pass@3 = 0.0
@sample 45 -> Pass@3 = 0.0
@sample 50 -> Pass@3 = 0.0
@sample 55 -> Pass@3 = 0.0
@sample 60 -> Pass@3 = 0.0
@sample 65 -> Pass@3 = 0.0
@sample 70 -> Pass@3 = 0.0
@sample 75 -> Pass@3 = 0.0
@sample 80 -> Pass@3 = 0.0
@sample 85 -> Pass@3 = 0.0
@sample 90 -> Pass@3 = 0.0
@sample 95 -> Pass@3 = 0.0
@sample 100 -> Pass@3 = 0.0


Pass@3 = 0.0



pass_at_k,▁
pass_at_k,0.0
